In [1]:
import numpy as np
import pandas as pd
import math
pd.set_option('display.max_columns', 100)
pd.options.display.float_format = '{:.2f}'.format

# Outline

###  Part A. Affordable  Rentals
 1. Bring in 2012 PUMS data
 2. Assign county_id to each puma
 3. Bring in and clean, reformat county AMIs data
 4. Add variables for cost limits for ELI, VLI, LI, MI, and HI by bedroom count for each county
 5. Add count dummies for each unit based on income and unit type
 

###  Part B. Affordable Owning
 1. Calculate total monthly housing cost by adding monthly first mortgage, second/junior mortgage, and condo fees
 2. Add count dummies for each unit based on income and unit type

### Part C. Weighting Sample to get estimates
 1. Multiply each dummy by the the household weight variable
 2. Calculte standard errors for each estimate
 
### Part D. Aggregating at PUMA Level
 1. Add dummy variables for vacant "for-rent," vacant "for-sale," and recently moved in and create second dataset
 2. Aggregate at PUMA level for each dataset and export

## Part A1. Bring in 2012 PUMS data

In [2]:
variable_types = {"BDSP":"float","RMSP":"str","MV":"float","WGTP":"float","TAXP":"float"}
columns = ["PUMA","RT","BDSP","BLD","RNTP","MRGP","SMP","CONP","TAXP","TEN","VACS","VALP","GRPIP","GRNTP","HINCP","MV","WGTP"]
pums_df=pd.read_csv("2012_pums.csv", delimiter=",",usecols=columns,dtype=variable_types)
pums_df

,RT,PUMA,WGTP,BDSP,BLD,CONP,MRGP,RNTP,SMP,TEN,VACS,VALP,GRNTP,GRPIP,HINCP,MV,TAXP
0,H,8508,110.00,4.00,2.00,nan,2300.00,nan,nan,1.00,nan,975000.00,nan,nan,105000.00,7.00,42.00
1,H,7502,180.00,1.00,9.00,nan,nan,nan,nan,nan,4.00,800000.00,nan,nan,nan,nan,nan
2,H,8105,75.00,2.00,7.00,nan,nan,1500.00,nan,3.00,nan,nan,1560.00,47.00,39600.00,4.00,nan
3,H,1305,76.00,4.00,2.00,nan,4800.00,nan,nan,1.00,nan,1000000.00,nan,nan,269000.00,4.00,68.00
4,H,2901,12.00,2.00,1.00,nan,690.00,nan,nan,1.00,nan,30000.00,nan,nan,65200.00,5.00,6.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153110,H,2901,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2.00,nan
153111,H,5500,34.00,4.00,2.00,nan,nan,nan,nan,2.00,nan,600000.00,nan,nan,72300.00,7.00,36.00
153112,H,300,192.00,2.00,5.00,nan,nan,nan,nan,nan,5.00,nan,nan,nan,nan,nan,nan
153113,H,5907,134.00,2.00,2.00,nan,3200.00,nan,120.00,1.00,nan,600000.00,nan,nan,242000.00,5.00,51.00


In [3]:
pums_df.rename(columns={"RT":"record_type","BDSP":"number_bedrooms",
    "BLD":"units_in_structure",
    "RNTP":"monthly_rent",
    "MRGP":"first_mortgage",
    "SMP":"second_mortgage",
    "CONP":"condo_fee",
    "TEN":"tenure",
    "VACS":"vacancy_status",
    "VALP":"property_value",
    "GRPIP":"gross_rent_pct_of_income",
    "GRNTP":"gross_rent",
    "HINCP":"hh_income",
    "TAXP":"prop_tax",
    "WGTP":"weight",
    "MV":"moved_in"}, inplace=True)
pums_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax
0,H,8508,110.00,4.00,2.00,nan,2300.00,nan,nan,1.00,nan,975000.00,nan,nan,105000.00,7.00,42.00
1,H,7502,180.00,1.00,9.00,nan,nan,nan,nan,nan,4.00,800000.00,nan,nan,nan,nan,nan
2,H,8105,75.00,2.00,7.00,nan,nan,1500.00,nan,3.00,nan,nan,1560.00,47.00,39600.00,4.00,nan
3,H,1305,76.00,4.00,2.00,nan,4800.00,nan,nan,1.00,nan,1000000.00,nan,nan,269000.00,4.00,68.00
4,H,2901,12.00,2.00,1.00,nan,690.00,nan,nan,1.00,nan,30000.00,nan,nan,65200.00,5.00,6.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153110,H,2901,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2.00,nan
153111,H,5500,34.00,4.00,2.00,nan,nan,nan,nan,2.00,nan,600000.00,nan,nan,72300.00,7.00,36.00
153112,H,300,192.00,2.00,5.00,nan,nan,nan,nan,nan,5.00,nan,nan,nan,nan,nan,nan
153113,H,5907,134.00,2.00,2.00,nan,3200.00,nan,120.00,1.00,nan,600000.00,nan,nan,242000.00,5.00,51.00


## Part A2. Assign county id to each puma

In [4]:
# load in crosswalk file
crosswalk_df=pd.read_csv("C:/Users/Terner GSR/Box/Cost of Doing Nothing/Climate Impacts/Data/PUMS/PUMA_County_Crosswalk_v2.csv", delimiter=",")
crosswalk_df

,PUMA,county1,county2,county3,county4,county5,county6,county7
0,101,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
1,102,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
2,103,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
3,104,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
4,105,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
260,11103,Ventura CA,NaN,NaN,NaN,NaN,NaN,NaN
261,11104,Ventura CA,NaN,NaN,NaN,NaN,NaN,NaN
262,11105,Ventura CA,NaN,NaN,NaN,NaN,NaN,NaN
263,11106,Ventura CA,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# add county name column to puma file
puma_county_df=pums_df.merge(crosswalk_df, how='left', left_on = "PUMA",right_on = "PUMA")
puma_county_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7
0,H,8508,110.00,4.00,2.00,nan,2300.00,nan,nan,1.00,nan,975000.00,nan,nan,105000.00,7.00,42.00,Santa Clara CA,NaN,NaN,NaN,NaN,NaN,NaN
1,H,7502,180.00,1.00,9.00,nan,nan,nan,nan,nan,4.00,800000.00,nan,nan,nan,nan,nan,San Francisco CA,NaN,NaN,NaN,NaN,NaN,NaN
2,H,8105,75.00,2.00,7.00,nan,nan,1500.00,nan,3.00,nan,nan,1560.00,47.00,39600.00,4.00,nan,San Mateo CA,NaN,NaN,NaN,NaN,NaN,NaN
3,H,1305,76.00,4.00,2.00,nan,4800.00,nan,nan,1.00,nan,1000000.00,nan,nan,269000.00,4.00,68.00,Contra Costa CA,NaN,NaN,NaN,NaN,NaN,NaN
4,H,2901,12.00,2.00,1.00,nan,690.00,nan,nan,1.00,nan,30000.00,nan,nan,65200.00,5.00,6.00,Kern CA,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153110,H,2901,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2.00,nan,Kern CA,NaN,NaN,NaN,NaN,NaN,NaN
153111,H,5500,34.00,4.00,2.00,nan,nan,nan,nan,2.00,nan,600000.00,nan,nan,72300.00,7.00,36.00,Napa CA,NaN,NaN,NaN,NaN,NaN,NaN
153112,H,300,192.00,2.00,5.00,nan,nan,nan,nan,nan,5.00,nan,nan,nan,nan,nan,nan,Alpine CA,Amador CA,Calaveras CA,Inyo CA,Mariposa CA,Mono CA,Tuolumne CA
153113,H,5907,134.00,2.00,2.00,nan,3200.00,nan,120.00,1.00,nan,600000.00,nan,nan,242000.00,5.00,51.00,Orange CA,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# remove " CA" from end of each county name
puma_county_df['County'] = puma_county_df['county1'].str.replace(r' CA', '')
puma_county_df['county2'] = puma_county_df['county2'].str.replace(r' CA', '')
puma_county_df['county3'] = puma_county_df['county3'].str.replace(r' CA', '')
puma_county_df['county4'] = puma_county_df['county4'].str.replace(r' CA', '')
puma_county_df['county5'] = puma_county_df['county5'].str.replace(r' CA', '')
puma_county_df['county6'] = puma_county_df['county6'].str.replace(r' CA', '')
puma_county_df['county7'] = puma_county_df['county7'].str.replace(r' CA', '')
puma_county_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County
0,H,8508,110.00,4.00,2.00,nan,2300.00,nan,nan,1.00,nan,975000.00,nan,nan,105000.00,7.00,42.00,Santa Clara CA,NaN,NaN,NaN,NaN,NaN,NaN,Santa Clara
1,H,7502,180.00,1.00,9.00,nan,nan,nan,nan,nan,4.00,800000.00,nan,nan,nan,nan,nan,San Francisco CA,NaN,NaN,NaN,NaN,NaN,NaN,San Francisco
2,H,8105,75.00,2.00,7.00,nan,nan,1500.00,nan,3.00,nan,nan,1560.00,47.00,39600.00,4.00,nan,San Mateo CA,NaN,NaN,NaN,NaN,NaN,NaN,San Mateo
3,H,1305,76.00,4.00,2.00,nan,4800.00,nan,nan,1.00,nan,1000000.00,nan,nan,269000.00,4.00,68.00,Contra Costa CA,NaN,NaN,NaN,NaN,NaN,NaN,Contra Costa
4,H,2901,12.00,2.00,1.00,nan,690.00,nan,nan,1.00,nan,30000.00,nan,nan,65200.00,5.00,6.00,Kern CA,NaN,NaN,NaN,NaN,NaN,NaN,Kern
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153110,H,2901,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2.00,nan,Kern CA,NaN,NaN,NaN,NaN,NaN,NaN,Kern
153111,H,5500,34.00,4.00,2.00,nan,nan,nan,nan,2.00,nan,600000.00,nan,nan,72300.00,7.00,36.00,Napa CA,NaN,NaN,NaN,NaN,NaN,NaN,Napa
153112,H,300,192.00,2.00,5.00,nan,nan,nan,nan,nan,5.00,nan,nan,nan,nan,nan,nan,Alpine CA,Amador,Calaveras,Inyo,Mariposa,Mono,Tuolumne,Alpine
153113,H,5907,134.00,2.00,2.00,nan,3200.00,nan,120.00,1.00,nan,600000.00,nan,nan,242000.00,5.00,51.00,Orange CA,NaN,NaN,NaN,NaN,NaN,NaN,Orange


## Part A3. Bring in and clean, reformat county AMIs data

In [7]:
#bring in 2018 county AMIs file
columns=["County","Income_Category","1","2","4","6","8"]
df = pd.read_csv("2012_amis.csv", delimiter=",", usecols=columns)
df

,County,Income_Category,1,2,4,6,8
0,Alameda County\n4-Per,Extremely Low,19650,22450,28050,32550,37050
1,Alameda County\n4-Per,Very Low Income,32750,37400,46750,54250,61750
2,Alameda County\n4-Per,Lower Income,45750,52300,65350,75850,86300
3,Alameda County\n4-Per,Median Income,65450,74800,93500,108450,123400
4,Alameda County\n4-Per,Moderate Income,78550,89750,112200,130150,148100
...,...,...,...,...,...,...,...
285,Yuba County\n4-Person,Extremely Low,12500,14250,17800,20650,23500
286,Yuba County\n4-Person,Very Low Income,20800,23800,29700,34500,39250
287,Yuba County\n4-Person,Lower Income,33250,38000,47500,55100,62700
288,Yuba County\n4-Person,Median Income,41600,47500,59400,68900,78400


In [8]:
#clean county names variable
df['County'] = df['County'].str.replace(r' County\n4-Person', '')
df['County'] = df['County'].str.replace(r' County\n4-Perso', '')
df['County'] = df['County'].str.replace(r' County\n4-Pers', '')
df['County'] = df['County'].str.replace(r' County\n4-Per', '')
df['County'] = df['County'].str.replace(r' County\n4-Pe', '')
df['County'] = df['County'].str.replace(r' County\n4-P', '')
df['County'] = df['County'].str.replace(r' County\n4-', '')
df['County'] = df['County'].str.replace(r' County\n4', '')
df['County'] = df['County'].str.replace(r' County\n', '')
df['County'] = df['County'].str.replace(r' County\ ' , '')
df['County'] = df['County'].str.replace(r' County', '')
df['County'] = df['County'].str.replace(r' Count', '')
df['County'] = df['County'].str.replace(r' Coun', '')
df

,County,Income_Category,1,2,4,6,8
0,Alameda,Extremely Low,19650,22450,28050,32550,37050
1,Alameda,Very Low Income,32750,37400,46750,54250,61750
2,Alameda,Lower Income,45750,52300,65350,75850,86300
3,Alameda,Median Income,65450,74800,93500,108450,123400
4,Alameda,Moderate Income,78550,89750,112200,130150,148100
...,...,...,...,...,...,...,...
285,Yuba,Extremely Low,12500,14250,17800,20650,23500
286,Yuba,Very Low Income,20800,23800,29700,34500,39250
287,Yuba,Lower Income,33250,38000,47500,55100,62700
288,Yuba,Median Income,41600,47500,59400,68900,78400


In [9]:
#create df for each income level and calculate max rents, rename variables
eli_df = df[df.Income_Category =="Extremely Low"].copy()
eli_df["1"]=eli_df["1"]*0.025
eli_df["2"]=eli_df["2"]*0.025
eli_df["4"]=eli_df["4"]*0.025
eli_df["6"]=eli_df["6"]*0.025
eli_df["8"]=eli_df["8"]*0.025
eli_df.rename(columns={"1":"ELI_studio","2":"ELI_1_br","4":"ELI_2_br","6":"ELI_3_br","8":"ELI_4_br"}, inplace=True)
eli_df.drop('Income_Category', axis=1, inplace=True)
eli_df

,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br
0,Alameda,491.25,561.25,701.25,813.75,926.25
5,Alpine,390.00,445.00,556.25,646.25,735.00
10,Amador,356.25,406.25,507.50,588.75,670.00
15,Butte,308.75,352.50,440.00,511.25,581.25
20,Calaveras,366.25,418.75,522.50,606.25,690.00
25,Colusa,303.75,347.50,433.75,503.75,573.75
30,Contra Costa,491.25,561.25,701.25,813.75,926.25
35,Del Norte,303.75,347.50,433.75,503.75,573.75
40,El Dorado,400.00,457.50,571.25,663.75,755.00
45,Fresno,303.75,347.50,433.75,503.75,573.75


In [10]:
vli_df = df[df.Income_Category =="Very Low Income"].copy()
vli_df["1"]=vli_df["1"]*0.025
vli_df["2"]=vli_df["2"]*0.025
vli_df["4"]=vli_df["4"]*0.025
vli_df["6"]=vli_df["6"]*0.025
vli_df["8"]=vli_df["8"]*0.025
vli_df.rename(columns={"1":"VLI_studio","2":"VLI_1_br","4":"VLI_2_br","6":"VLI_3_br","8":"VLI_4_br"}, inplace=True)
vli_df.drop('Income_Category', axis=1, inplace=True)
vli_df

,County,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br
1,Alameda,818.75,935.00,1168.75,1356.25,1543.75
6,Alpine,648.75,741.25,926.25,1075.00,1223.75
11,Amador,592.50,677.50,846.25,982.50,1117.50
16,Butte,513.75,587.50,733.75,851.25,968.75
21,Calaveras,610.00,697.50,871.25,1011.25,1151.25
26,Colusa,507.50,580.00,723.75,840.00,956.25
31,Contra Costa,818.75,935.00,1168.75,1356.25,1543.75
36,Del Norte,507.50,580.00,723.75,840.00,956.25
41,El Dorado,666.25,761.25,951.25,1103.75,1256.25
46,Fresno,507.50,580.00,723.75,840.00,956.25


In [11]:
li_df = df[df.Income_Category =="Lower Income"].copy()
li_df["1"]=li_df["1"]*0.025
li_df["2"]=li_df["2"]*0.025
li_df["4"]=li_df["4"]*0.025
li_df["6"]=li_df["6"]*0.025
li_df["8"]=li_df["8"]*0.025
li_df.rename(columns={"1":"LI_studio","2":"LI_1_br","4":"LI_2_br","6":"LI_3_br","8":"LI_4_br"}, inplace=True)
li_df.drop('Income_Category', axis=1, inplace=True)
li_df

,County,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br
2,Alameda,1143.75,1307.50,1633.75,1896.25,2157.50
7,Alpine,1038.75,1186.25,1482.50,1720.00,1957.50
12,Amador,948.75,1083.75,1353.75,1571.25,1787.50
17,Butte,822.50,940.00,1173.75,1362.50,1550.00
22,Calaveras,976.25,1115.00,1393.75,1617.50,1840.00
27,Colusa,811.25,926.25,1157.50,1343.75,1528.75
32,Contra Costa,1143.75,1307.50,1633.75,1896.25,2157.50
37,Del Norte,811.25,926.25,1157.50,1343.75,1528.75
42,El Dorado,1066.25,1218.75,1522.50,1766.25,2010.00
47,Fresno,811.25,926.25,1157.50,1343.75,1528.75


In [12]:
mi_df = df[df.Income_Category =="Median Income"].copy()
mi_df["1"]=mi_df["1"]*0.025
mi_df["2"]=mi_df["2"]*0.025
mi_df["4"]=mi_df["4"]*0.025
mi_df["6"]=mi_df["6"]*0.025
mi_df["8"]=mi_df["8"]*0.025
mi_df.rename(columns={"1":"MI_studio","2":"MI_1_br","4":"MI_2_br","6":"MI_3_br","8":"MI_4_br"}, inplace=True)
mi_df.drop('Income_Category', axis=1, inplace=True)
mi_df

,County,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br
3,Alameda,1636.25,1870.00,2337.50,2711.25,3085.00
8,Alpine,1483.75,1696.25,2120.00,2458.75,2798.75
13,Amador,1185.00,1353.75,1692.50,1963.75,2233.75
18,Butte,1027.50,1173.75,1467.50,1702.50,1937.50
23,Calaveras,1220.00,1393.75,1742.50,2021.25,2300.00
28,Colusa,1013.75,1157.50,1447.50,1678.75,1911.25
33,Contra Costa,1636.25,1870.00,2337.50,2711.25,3085.00
38,Del Norte,1013.75,1157.50,1447.50,1678.75,1911.25
43,El Dorado,1331.25,1522.50,1902.50,2207.50,2511.25
48,Fresno,1013.75,1157.50,1447.50,1678.75,1911.25


In [13]:
moi_df = df[df.Income_Category =="Moderate Income"].copy()
moi_df["1"]=moi_df["1"]*0.025
moi_df["2"]=moi_df["2"]*0.025
moi_df["4"]=moi_df["4"]*0.025
moi_df["6"]=moi_df["6"]*0.025
moi_df["8"]=moi_df["8"]*0.025
moi_df.rename(columns={"1":"MoI_studio","2":"MoI_1_br","4":"MoI_2_br","6":"MoI_3_br","8":"MoI_4_br"}, inplace=True)
moi_df.drop('Income_Category', axis=1, inplace=True)
moi_df

,County,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br
4,Alameda,1963.75,2243.75,2805.00,3253.75,3702.50
9,Alpine,1781.25,2035.00,2543.75,2951.25,3357.50
14,Amador,1422.50,1625.00,2031.25,2356.25,2681.25
19,Butte,1232.50,1408.75,1761.25,2042.50,2325.00
24,Calaveras,1463.75,1672.50,2091.25,2426.25,2760.00
29,Colusa,1216.25,1390.00,1737.50,2015.00,2293.75
34,Contra Costa,1963.75,2243.75,2805.00,3253.75,3702.50
39,Del Norte,1216.25,1390.00,1737.50,2015.00,2293.75
44,El Dorado,1597.50,1826.25,2282.50,2647.50,3012.50
49,Fresno,1216.25,1390.00,1737.50,2015.00,2293.75


In [14]:
#combine dfs
all_amis_df=eli_df.merge(vli_df, how='left', left_on = "County",right_on = "County")
all_amis_df=all_amis_df.merge(li_df, how='left', left_on = "County",right_on = "County")
all_amis_df=all_amis_df.merge(mi_df, how='left', left_on = "County",right_on = "County")
all_amis_df=all_amis_df.merge(moi_df, how='left', left_on = "County",right_on = "County")
all_amis_df

,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br
0,Alameda,491.25,561.25,701.25,813.75,926.25,818.75,935.00,1168.75,1356.25,1543.75,1143.75,1307.50,1633.75,1896.25,2157.50,1636.25,1870.00,2337.50,2711.25,3085.00,1963.75,2243.75,2805.00,3253.75,3702.50
1,Alpine,390.00,445.00,556.25,646.25,735.00,648.75,741.25,926.25,1075.00,1223.75,1038.75,1186.25,1482.50,1720.00,1957.50,1483.75,1696.25,2120.00,2458.75,2798.75,1781.25,2035.00,2543.75,2951.25,3357.50
2,Amador,356.25,406.25,507.50,588.75,670.00,592.50,677.50,846.25,982.50,1117.50,948.75,1083.75,1353.75,1571.25,1787.50,1185.00,1353.75,1692.50,1963.75,2233.75,1422.50,1625.00,2031.25,2356.25,2681.25
3,Butte,308.75,352.50,440.00,511.25,581.25,513.75,587.50,733.75,851.25,968.75,822.50,940.00,1173.75,1362.50,1550.00,1027.50,1173.75,1467.50,1702.50,1937.50,1232.50,1408.75,1761.25,2042.50,2325.00
4,Calaveras,366.25,418.75,522.50,606.25,690.00,610.00,697.50,871.25,1011.25,1151.25,976.25,1115.00,1393.75,1617.50,1840.00,1220.00,1393.75,1742.50,2021.25,2300.00,1463.75,1672.50,2091.25,2426.25,2760.00
5,Colusa,303.75,347.50,433.75,503.75,573.75,507.50,580.00,723.75,840.00,956.25,811.25,926.25,1157.50,1343.75,1528.75,1013.75,1157.50,1447.50,1678.75,1911.25,1216.25,1390.00,1737.50,2015.00,2293.75
6,Contra Costa,491.25,561.25,701.25,813.75,926.25,818.75,935.00,1168.75,1356.25,1543.75,1143.75,1307.50,1633.75,1896.25,2157.50,1636.25,1870.00,2337.50,2711.25,3085.00,1963.75,2243.75,2805.00,3253.75,3702.50
7,Del Norte,303.75,347.50,433.75,503.75,573.75,507.50,580.00,723.75,840.00,956.25,811.25,926.25,1157.50,1343.75,1528.75,1013.75,1157.50,1447.50,1678.75,1911.25,1216.25,1390.00,1737.50,2015.00,2293.75
8,El Dorado,400.00,457.50,571.25,663.75,755.00,666.25,761.25,951.25,1103.75,1256.25,1066.25,1218.75,1522.50,1766.25,2010.00,1331.25,1522.50,1902.50,2207.50,2511.25,1597.50,1826.25,2282.50,2647.50,3012.50
9,Fresno,303.75,347.50,433.75,503.75,573.75,507.50,580.00,723.75,840.00,956.25,811.25,926.25,1157.50,1343.75,1528.75,1013.75,1157.50,1447.50,1678.75,1911.25,1216.25,1390.00,1737.50,2015.00,2293.75


## Part A4. Add count dummies for each unit based on income and unit type

In [15]:
# merge datasets
units_df=puma_county_df.merge(all_amis_df, how='left', left_on = "County",right_on = "County")
units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br
0,H,8508,110.00,4.00,2.00,nan,2300.00,nan,nan,1.00,nan,975000.00,nan,nan,105000.00,7.00,42.00,Santa Clara CA,NaN,NaN,NaN,NaN,NaN,NaN,Santa Clara,551.25,630.00,787.50,913.75,1040.00,918.75,1050.00,1312.50,1522.50,1732.50,1325.00,1515.00,1892.50,2196.25,2498.75,1837.50,2100.00,2625.00,3045.00,3465.00,2205.00,2520.00,3150.00,3653.75,4157.50
1,H,7502,180.00,1.00,9.00,nan,nan,nan,nan,nan,4.00,800000.00,nan,nan,nan,nan,nan,San Francisco CA,NaN,NaN,NaN,NaN,NaN,NaN,San Francisco,583.75,666.25,832.50,966.25,1100.00,971.25,1110.00,1387.50,1610.00,1832.50,1555.00,1776.25,2220.00,2576.25,2931.25,1802.50,2060.00,2575.00,2987.50,3398.75,2162.50,2472.50,3090.00,3585.00,4078.75
2,H,8105,75.00,2.00,7.00,nan,nan,1500.00,nan,3.00,nan,nan,1560.00,47.00,39600.00,4.00,nan,San Mateo CA,NaN,NaN,NaN,NaN,NaN,NaN,San Mateo,583.75,666.25,832.50,966.25,1100.00,971.25,1110.00,1387.50,1610.00,1832.50,1555.00,1776.25,2220.00,2576.25,2931.25,1802.50,2060.00,2575.00,2987.50,3398.75,2162.50,2472.50,3090.00,3585.00,4078.75
3,H,1305,76.00,4.00,2.00,nan,4800.00,nan,nan,1.00,nan,1000000.00,nan,nan,269000.00,4.00,68.00,Contra Costa CA,NaN,NaN,NaN,NaN,NaN,NaN,Contra Costa,491.25,561.25,701.25,813.75,926.25,818.75,935.00,1168.75,1356.25,1543.75,1143.75,1307.50,1633.75,1896.25,2157.50,1636.25,1870.00,2337.50,2711.25,3085.00,1963.75,2243.75,2805.00,3253.75,3702.50
4,H,2901,12.00,2.00,1.00,nan,690.00,nan,nan,1.00,nan,30000.00,nan,nan,65200.00,5.00,6.00,Kern CA,NaN,NaN,NaN,NaN,NaN,NaN,Kern,303.75,347.50,433.75,503.75,573.75,507.50,580.00,723.75,840.00,956.25,811.25,926.25,1157.50,1343.75,1528.75,1013.75,1157.50,1447.50,1678.75,1911.25,1216.25,1390.00,1737.50,2015.00,2293.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153110,H,2901,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2.00,nan,Kern CA,NaN,NaN,NaN,NaN,NaN,NaN,Kern,303.75,347.50,433.75,503.75,573.75,507.50,580.00,723.75,840.00,956.25,811.25,926.25,1157.50,1343.75,1528.75,1013.75,1157.50,1447.50,1678.75,1911.25,1216.25,1390.00,1737.50,2015.00,2293.75
153111,H,5500,34.00,4.00,2.00,nan,nan,nan,nan,2.00,nan,600000.00,nan,nan,72300.00,7.00,36.00,Napa CA,NaN,NaN,NaN,NaN,NaN,NaN,Napa,452.50,517.50,646.25,750.00,853.75,753.75,861.25,1076.25,1248.75,1421.25,1137.50,1300.00,1625.00,1885.00,2145.00,1506.25,1722.50,2152.50,2497.50,2841.25,1807.50,2066.25,2582.50,2996.25,3408.75
153112,H,300,192.00,2.00,5.00,nan,nan,nan,nan,nan,5.00,nan,nan,nan,nan,nan,nan,Alpine CA,Amador,Calaveras,Inyo,Mariposa,Mono,Tuolumne,Alpine,390.00,445.00,556.25,646.25,735.00,648.75,741.25,926.25,1075.00,1223.75,1038.75,1186.25,1482.50,1720.00,1957.50,1483.75,1696.25,2120.00,2458.75,2798.75,1781.25,2035.00,2543.75,2951.25,3357.50
153113,H,5907,134.00,2.00,2.00,nan,3200.00,nan,120.00,1.00,nan,600000.00,nan,nan,242000.00,5.00,51.00,Orange CA,NaN,NaN,NaN,NaN,NaN,NaN,Orange,506.25,578.75,722.50,838.75,953.75,843.75,963.75,1203.75,1397.50,1590.00,1348.75,1541.25,1926.25,2235.00,2543.75,1492.50,1706.25,2132.50,2473.75,2815.00,1791.25,2047.50,2558.75,2968.75,3377.50


In [16]:
# create filter masks for each affordability and unit type
ELI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['monthly_rent']<=units_df["ELI_studio"])
ELI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['monthly_rent']<=units_df["ELI_1_br"])
ELI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['monthly_rent']<=units_df["ELI_2_br"])
ELI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['monthly_rent']<=units_df["ELI_3_br"])
ELI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['monthly_rent']<=units_df["ELI_4_br"])
VLI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['monthly_rent']<=units_df["VLI_studio"])
VLI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['monthly_rent']<=units_df["VLI_1_br"])
VLI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['monthly_rent']<=units_df["VLI_2_br"])
VLI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['monthly_rent']<=units_df["VLI_3_br"])
VLI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['monthly_rent']<=units_df["VLI_4_br"])
LI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['monthly_rent']<=units_df["LI_studio"])
LI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['monthly_rent']<=units_df["LI_1_br"])
LI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['monthly_rent']<=units_df["LI_2_br"])
LI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['monthly_rent']<=units_df["LI_3_br"])
LI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['monthly_rent']<=units_df["LI_4_br"])
MI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['monthly_rent']<=units_df["MI_studio"])
MI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['monthly_rent']<=units_df["MI_1_br"])
MI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['monthly_rent']<=units_df["MI_2_br"])
MI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['monthly_rent']<=units_df["MI_3_br"])
MI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['monthly_rent']<=units_df["MI_4_br"])
MoI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['monthly_rent']<=units_df["MoI_studio"])
MoI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['monthly_rent']<=units_df["MoI_1_br"])
MoI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['monthly_rent']<=units_df["MoI_2_br"])
MoI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['monthly_rent']<=units_df["MoI_3_br"])
MoI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['monthly_rent']<=units_df["MoI_4_br"])
HI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['monthly_rent']>=units_df["MoI_studio"])
HI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['monthly_rent']>=units_df["MoI_1_br"])
HI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['monthly_rent']>=units_df["MoI_2_br"])
HI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['monthly_rent']>=units_df["MoI_3_br"])
HI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['monthly_rent']>=units_df["MoI_4_br"])

# apply filter masks to create dummy variables
units_df['rent_ELI_studio_count']=np.where(ELI_studio_mask,1,0)
units_df['rent_ELI_1_br_count']=np.where(ELI_1_br_mask,1,0)
units_df['rent_ELI_2_br_count']=np.where(ELI_2_br_mask,1,0)
units_df['rent_ELI_3_br_count']=np.where(ELI_3_br_mask,1,0)
units_df['rent_ELI_4_br_count']=np.where(ELI_4_br_mask,1,0)
units_df['rent_VLI_studio_count']=np.where(VLI_studio_mask,1,0)
units_df['rent_VLI_1_br_count']=np.where(VLI_1_br_mask,1,0)
units_df['rent_VLI_2_br_count']=np.where(VLI_2_br_mask,1,0)
units_df['rent_VLI_3_br_count']=np.where(VLI_3_br_mask,1,0)
units_df['rent_VLI_4_br_count']=np.where(VLI_4_br_mask,1,0)
units_df['rent_LI_studio_count']=np.where(LI_studio_mask,1,0)
units_df['rent_LI_1_br_count']=np.where(LI_1_br_mask,1,0)
units_df['rent_LI_2_br_count']=np.where(LI_2_br_mask,1,0)
units_df['rent_LI_3_br_count']=np.where(LI_3_br_mask,1,0)
units_df['rent_LI_4_br_count']=np.where(LI_4_br_mask,1,0)
units_df['rent_MI_studio_count']=np.where(MI_studio_mask,1,0)
units_df['rent_MI_1_br_count']=np.where(MI_1_br_mask,1,0)
units_df['rent_MI_2_br_count']=np.where(MI_2_br_mask,1,0)
units_df['rent_MI_3_br_count']=np.where(MI_3_br_mask,1,0)
units_df['rent_MI_4_br_count']=np.where(MI_4_br_mask,1,0)
units_df['rent_MoI_studio_count']=np.where(MoI_studio_mask,1,0)
units_df['rent_MoI_1_br_count']=np.where(MoI_1_br_mask,1,0)
units_df['rent_MoI_2_br_count']=np.where(MoI_2_br_mask,1,0)
units_df['rent_MoI_3_br_count']=np.where(MoI_3_br_mask,1,0)
units_df['rent_MoI_4_br_count']=np.where(MoI_4_br_mask,1,0)
units_df['rent_HI_studio_count']=np.where(HI_studio_mask,1,0)
units_df['rent_HI_1_br_count']=np.where(HI_1_br_mask,1,0)
units_df['rent_HI_2_br_count']=np.where(HI_2_br_mask,1,0)
units_df['rent_HI_3_br_count']=np.where(HI_3_br_mask,1,0)
units_df['rent_HI_4_br_count']=np.where(HI_4_br_mask,1,0)
units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,rent_ELI_studio_count,rent_ELI_1_br_count,rent_ELI_2_br_count,rent_ELI_3_br_count,rent_ELI_4_br_count,rent_VLI_studio_count,rent_VLI_1_br_count,rent_VLI_2_br_count,rent_VLI_3_br_count,rent_VLI_4_br_count,rent_LI_studio_count,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count
0,H,8508,110.00,4.00,2.00,nan,2300.00,nan,nan,1.00,nan,975000.00,nan,nan,105000.00,7.00,42.00,Santa Clara CA,NaN,NaN,NaN,NaN,NaN,NaN,Santa Clara,551.25,630.00,787.50,913.75,1040.00,918.75,1050.00,1312.50,1522.50,1732.50,1325.00,1515.00,1892.50,2196.25,2498.75,1837.50,2100.00,2625.00,3045.00,3465.00,2205.00,2520.00,3150.00,3653.75,4157.50,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,H,7502,180.00,1.00,9.00,nan,nan,nan,nan,nan,4.00,800000.00,nan,nan,nan,nan,nan,San Francisco CA,NaN,NaN,NaN,NaN,NaN,NaN,San Francisco,583.75,666.25,832.50,966.25,1100.00,971.25,1110.00,1387.50,1610.00,1832.50,1555.00,1776.25,2220.00,2576.25,2931.25,1802.50,2060.00,2575.00,2987.50,3398.75,2162.50,2472.50,3090.00,3585.00,4078.75,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,H,8105,75.00,2.00,7.00,nan,nan,1500.00,nan,3.00,nan,nan,1560.00,47.00,39600.00,4.00,nan,San Mateo CA,NaN,NaN,NaN,NaN,NaN,NaN,San Mateo,583.75,666.25,832.50,966.25,1100.00,971.25,1110.00,1387.50,1610.00,1832.50,1555.00,1776.25,2220.00,2576.25,2931.25,1802.50,2060.00,2575.00,2987.50,3398.75,2162.50,2472.50,3090.00,3585.00,4078.75,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0
3,H,1305,76.00,4.00,2.00,nan,4800.00,nan,nan,1.00,nan,1000000.00,nan,nan,269000.00,4.00,68.00,Contra Costa CA,NaN,NaN,NaN,NaN,NaN,NaN,Contra Costa,491.25,561.25,701.25,813.75,926.25,818.75,935.00,1168.75,1356.25,1543.75,1143.75,1307.50,1633.75,1896.25,2157.50,1636.25,1870.00,2337.50,2711.25,3085.00,1963.75,2243.75,2805.00,3253.75,3702.50,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,H,2901,12.00,2.00,1.00,nan,690.00,nan,nan,1.00,nan,30000.00,nan,nan,65200.00,5.00,6.00,Kern CA,NaN,NaN,NaN,NaN,NaN,NaN,Kern,303.75,347.50,433.75,503.75,573.75,507.50,580.00,723.75,840.00,956.25,811.25,926.25,1157.50,1343.75,1528.75,1013.75,1157.50,1447.50,1678.75,1911.25,1216.25,1390.00,1737.50,2015.00,2293.75,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153110,H,2901,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2.00,nan,Kern CA,NaN,NaN,NaN,NaN,NaN,NaN,Kern,303.75,347.50,433.75,503.75,573.75,507.50,580.00,723.75,840.00,956.25,811.25,926.25,1157.50,1343.75,1528.75,1013.75,1157.50,1447.50,1678.75,1911.25,1216.25,1390.00,1737.50,2015.00,2293.75,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
153111,H,5500,34.00,4.00,2.00,nan,nan,nan,nan,2.00,nan,600000.00,nan,nan,72300.00,7.00,36.00,Napa CA,NaN,NaN,NaN,NaN,NaN,NaN,Napa,452.50,517.50,646.25,750.00,853.75,753.75,861.25,1076.25,1248.75,1421.25,1137.50,1300.00,1625.00,1885.00,2145.00,1506.25,1722.50

## Part B1. Calculate total monthly housing cost by adding monthly first mortgage, second/junior mortgage, and condo fees

In [17]:
units_df['own_monthly_cost']=units_df['first_mortgage'] + units_df['second_mortgage'] + units_df['condo_fee'] + (units_df['prop_tax']/12)
units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,rent_ELI_studio_count,rent_ELI_1_br_count,rent_ELI_2_br_count,rent_ELI_3_br_count,rent_ELI_4_br_count,rent_VLI_studio_count,rent_VLI_1_br_count,rent_VLI_2_br_count,rent_VLI_3_br_count,rent_VLI_4_br_count,rent_LI_studio_count,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_monthly_cost
0,H,8508,110.00,4.00,2.00,nan,2300.00,nan,nan,1.00,nan,975000.00,nan,nan,105000.00,7.00,42.00,Santa Clara CA,NaN,NaN,NaN,NaN,NaN,NaN,Santa Clara,551.25,630.00,787.50,913.75,1040.00,918.75,1050.00,1312.50,1522.50,1732.50,1325.00,1515.00,1892.50,2196.25,2498.75,1837.50,2100.00,2625.00,3045.00,3465.00,2205.00,2520.00,3150.00,3653.75,4157.50,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan
1,H,7502,180.00,1.00,9.00,nan,nan,nan,nan,nan,4.00,800000.00,nan,nan,nan,nan,nan,San Francisco CA,NaN,NaN,NaN,NaN,NaN,NaN,San Francisco,583.75,666.25,832.50,966.25,1100.00,971.25,1110.00,1387.50,1610.00,1832.50,1555.00,1776.25,2220.00,2576.25,2931.25,1802.50,2060.00,2575.00,2987.50,3398.75,2162.50,2472.50,3090.00,3585.00,4078.75,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan
2,H,8105,75.00,2.00,7.00,nan,nan,1500.00,nan,3.00,nan,nan,1560.00,47.00,39600.00,4.00,nan,San Mateo CA,NaN,NaN,NaN,NaN,NaN,NaN,San Mateo,583.75,666.25,832.50,966.25,1100.00,971.25,1110.00,1387.50,1610.00,1832.50,1555.00,1776.25,2220.00,2576.25,2931.25,1802.50,2060.00,2575.00,2987.50,3398.75,2162.50,2472.50,3090.00,3585.00,4078.75,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,nan
3,H,1305,76.00,4.00,2.00,nan,4800.00,nan,nan,1.00,nan,1000000.00,nan,nan,269000.00,4.00,68.00,Contra Costa CA,NaN,NaN,NaN,NaN,NaN,NaN,Contra Costa,491.25,561.25,701.25,813.75,926.25,818.75,935.00,1168.75,1356.25,1543.75,1143.75,1307.50,1633.75,1896.25,2157.50,1636.25,1870.00,2337.50,2711.25,3085.00,1963.75,2243.75,2805.00,3253.75,3702.50,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan
4,H,2901,12.00,2.00,1.00,nan,690.00,nan,nan,1.00,nan,30000.00,nan,nan,65200.00,5.00,6.00,Kern CA,NaN,NaN,NaN,NaN,NaN,NaN,Kern,303.75,347.50,433.75,503.75,573.75,507.50,580.00,723.75,840.00,956.25,811.25,926.25,1157.50,1343.75,1528.75,1013.75,1157.50,1447.50,1678.75,1911.25,1216.25,1390.00,1737.50,2015.00,2293.75,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153110,H,2901,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2.00,nan,Kern CA,NaN,NaN,NaN,NaN,NaN,NaN,Kern,303.75,347.50,433.75,503.75,573.75,507.50,580.00,723.75,840.00,956.25,811.25,926.25,1157.50,1343.75,1528.75,1013.75,1157.50,1447.50,1678.75,1911.25,1216.25,1390.00,1737.50,2015.00,2293.75,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan
153111,H,5500,34.00,4.00,2.00,nan,nan,nan,nan,2.00,nan,600000.00,nan,nan,72300.00,7.00,36.00,Napa CA,NaN,NaN,NaN,NaN,NaN,NaN,Napa,452.50,517.50,646.25,750.00,853.75,753.75,861.25,1076.25,1248.75,1421.25,1137.50,13

# Part B2. Add count dummies for each unit based on income and unit type

In [18]:
# create filter masks for each affordability and unit type
ELI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['own_monthly_cost']<=units_df["ELI_studio"])
ELI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['own_monthly_cost']<=units_df["ELI_1_br"])
ELI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['own_monthly_cost']<=units_df["ELI_2_br"])
ELI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['own_monthly_cost']<=units_df["ELI_3_br"])
ELI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['own_monthly_cost']<=units_df["ELI_4_br"])
VLI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['own_monthly_cost']<=units_df["VLI_studio"])
VLI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['own_monthly_cost']<=units_df["VLI_1_br"])
VLI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['own_monthly_cost']<=units_df["VLI_2_br"])
VLI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['own_monthly_cost']<=units_df["VLI_3_br"])
VLI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['own_monthly_cost']<=units_df["VLI_4_br"])
LI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['own_monthly_cost']<=units_df["LI_studio"])
LI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['own_monthly_cost']<=units_df["LI_1_br"])
LI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['own_monthly_cost']<=units_df["LI_2_br"])
LI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['own_monthly_cost']<=units_df["LI_3_br"])
LI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['own_monthly_cost']<=units_df["LI_4_br"])
MI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['own_monthly_cost']<=units_df["MI_studio"])
MI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['own_monthly_cost']<=units_df["MI_1_br"])
MI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['own_monthly_cost']<=units_df["MI_2_br"])
MI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['own_monthly_cost']<=units_df["MI_3_br"])
MI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['own_monthly_cost']<=units_df["MI_4_br"])
MoI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['own_monthly_cost']<=units_df["MoI_studio"])
MoI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['own_monthly_cost']<=units_df["MoI_1_br"])
MoI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['own_monthly_cost']<=units_df["MoI_2_br"])
MoI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['own_monthly_cost']<=units_df["MoI_3_br"])
MoI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['own_monthly_cost']<=units_df["MoI_4_br"])
HI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['own_monthly_cost']>=units_df["MoI_studio"])
HI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['own_monthly_cost']>=units_df["MoI_1_br"])
HI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['own_monthly_cost']>=units_df["MoI_2_br"])
HI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['own_monthly_cost']>=units_df["MoI_3_br"])
HI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['own_monthly_cost']>=units_df["MoI_4_br"])

# apply filter masks to create dummy variables
units_df['own_ELI_studio_count']=np.where(ELI_studio_mask,1,0)
units_df['own_ELI_1_br_count']=np.where(ELI_1_br_mask,1,0)
units_df['own_ELI_2_br_count']=np.where(ELI_2_br_mask,1,0)
units_df['own_ELI_3_br_count']=np.where(ELI_3_br_mask,1,0)
units_df['own_ELI_4_br_count']=np.where(ELI_4_br_mask,1,0)
units_df['own_VLI_studio_count']=np.where(VLI_studio_mask,1,0)
units_df['own_VLI_1_br_count']=np.where(VLI_1_br_mask,1,0)
units_df['own_VLI_2_br_count']=np.where(VLI_2_br_mask,1,0)
units_df['own_VLI_3_br_count']=np.where(VLI_3_br_mask,1,0)
units_df['own_VLI_4_br_count']=np.where(VLI_4_br_mask,1,0)
units_df['own_LI_studio_count']=np.where(LI_studio_mask,1,0)
units_df['own_LI_1_br_count']=np.where(LI_1_br_mask,1,0)
units_df['own_LI_2_br_count']=np.where(LI_2_br_mask,1,0)
units_df['own_LI_3_br_count']=np.where(LI_3_br_mask,1,0)
units_df['own_LI_4_br_count']=np.where(LI_4_br_mask,1,0)
units_df['own_MI_studio_count']=np.where(MI_studio_mask,1,0)
units_df['own_MI_1_br_count']=np.where(MI_1_br_mask,1,0)
units_df['own_MI_2_br_count']=np.where(MI_2_br_mask,1,0)
units_df['own_MI_3_br_count']=np.where(MI_3_br_mask,1,0)
units_df['own_MI_4_br_count']=np.where(MI_4_br_mask,1,0)
units_df['own_MoI_studio_count']=np.where(MoI_studio_mask,1,0)
units_df['own_MoI_1_br_count']=np.where(MoI_1_br_mask,1,0)
units_df['own_MoI_2_br_count']=np.where(MoI_2_br_mask,1,0)
units_df['own_MoI_3_br_count']=np.where(MoI_3_br_mask,1,0)
units_df['own_MoI_4_br_count']=np.where(MoI_4_br_mask,1,0)
units_df['own_HI_studio_count']=np.where(HI_studio_mask,1,0)
units_df['own_HI_1_br_count']=np.where(HI_1_br_mask,1,0)
units_df['own_HI_2_br_count']=np.where(HI_2_br_mask,1,0)
units_df['own_HI_3_br_count']=np.where(HI_3_br_mask,1,0)
units_df['own_HI_4_br_count']=np.where(HI_4_br_mask,1,0)
units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,...,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_monthly_cost,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count
0,H,8508,110.00,4.00,2.00,nan,2300.00,nan,nan,1.00,nan,975000.00,nan,nan,105000.00,7.00,42.00,Santa Clara CA,NaN,NaN,NaN,NaN,NaN,NaN,Santa Clara,551.25,630.00,787.50,913.75,1040.00,918.75,1050.00,1312.50,1522.50,1732.50,1325.00,1515.00,1892.50,2196.25,2498.75,1837.50,2100.00,2625.00,3045.00,3465.00,2205.00,2520.00,3150.00,3653.75,4157.50,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,H,7502,180.00,1.00,9.00,nan,nan,nan,nan,nan,4.00,800000.00,nan,nan,nan,nan,nan,San Francisco CA,NaN,NaN,NaN,NaN,NaN,NaN,San Francisco,583.75,666.25,832.50,966.25,1100.00,971.25,1110.00,1387.50,1610.00,1832.50,1555.00,1776.25,2220.00,2576.25,2931.25,1802.50,2060.00,2575.00,2987.50,3398.75,2162.50,2472.50,3090.00,3585.00,4078.75,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,H,8105,75.00,2.00,7.00,nan,nan,1500.00,nan,3.00,nan,nan,1560.00,47.00,39600.00,4.00,nan,San Mateo CA,NaN,NaN,NaN,NaN,NaN,NaN,San Mateo,583.75,666.25,832.50,966.25,1100.00,971.25,1110.00,1387.50,1610.00,1832.50,1555.00,1776.25,2220.00,2576.25,2931.25,1802.50,2060.00,2575.00,2987.50,3398.75,2162.50,2472.50,3090.00,3585.00,4078.75,...,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,H,1305,76.00,4.00,2.00,nan,4800.00,nan,nan,1.00,nan,1000000.00,nan,nan,269000.00,4.00,68.00,Contra Costa CA,NaN,NaN,NaN,NaN,NaN,NaN,Contra Costa,491.25,561.25,701.25,813.75,926.25,818.75,935.00,1168.75,1356.25,1543.75,1143.75,1307.50,1633.75,1896.25,2157.50,1636.25,1870.00,2337.50,2711.25,3085.00,1963.75,2243.75,2805.00,3253.75,3702.50,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,H,2901,12.00,2.00,1.00,nan,690.00,nan,nan,1.00,nan,30000.00,nan,nan,65200.00,5.00,6.00,Kern CA,NaN,NaN,NaN,NaN,NaN,NaN,Kern,303.75,347.50,433.75,503.75,573.75,507.50,580.00,723.75,840.00,956.25,811.25,926.25,1157.50,1343.75,1528.75,1013.75,1157.50,1447.50,1678.75,1911.25,1216.25,1390.00,1737.50,2015.00,2293.75,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

# Part C1. Multiply each dummy by the the household weight variable

In [19]:
units_df.rent_ELI_studio_count=units_df.rent_ELI_studio_count*units_df.weight
units_df.rent_ELI_1_br_count=units_df.rent_ELI_1_br_count*units_df.weight
units_df.rent_ELI_2_br_count=units_df.rent_ELI_2_br_count*units_df.weight
units_df.rent_ELI_3_br_count=units_df.rent_ELI_3_br_count*units_df.weight
units_df.rent_ELI_4_br_count=units_df.rent_ELI_4_br_count*units_df.weight
units_df.rent_VLI_studio_count=units_df.rent_VLI_studio_count*units_df.weight
units_df.rent_VLI_1_br_count=units_df.rent_VLI_1_br_count*units_df.weight
units_df.rent_VLI_2_br_count=units_df.rent_VLI_2_br_count*units_df.weight
units_df.rent_VLI_3_br_count=units_df.rent_VLI_3_br_count*units_df.weight
units_df.rent_VLI_4_br_count=units_df.rent_VLI_4_br_count*units_df.weight
units_df.rent_LI_studio_count=units_df.rent_LI_studio_count*units_df.weight
units_df.rent_LI_1_br_count=units_df.rent_LI_1_br_count*units_df.weight
units_df.rent_LI_2_br_count=units_df.rent_LI_2_br_count*units_df.weight
units_df.rent_LI_3_br_count=units_df.rent_LI_3_br_count*units_df.weight
units_df.rent_LI_4_br_count=units_df.rent_LI_4_br_count*units_df.weight
units_df.rent_MI_studio_count=units_df.rent_MI_studio_count*units_df.weight
units_df.rent_MI_1_br_count=units_df.rent_MI_1_br_count*units_df.weight
units_df.rent_MI_2_br_count=units_df.rent_MI_2_br_count*units_df.weight
units_df.rent_MI_3_br_count=units_df.rent_MI_3_br_count*units_df.weight
units_df.rent_MI_4_br_count=units_df.rent_MI_4_br_count*units_df.weight
units_df.rent_MoI_studio_count=units_df.rent_MoI_studio_count*units_df.weight
units_df.rent_MoI_1_br_count=units_df.rent_MoI_1_br_count*units_df.weight
units_df.rent_MoI_2_br_count=units_df.rent_MoI_2_br_count*units_df.weight
units_df.rent_MoI_3_br_count=units_df.rent_MoI_3_br_count*units_df.weight
units_df.rent_MoI_4_br_count=units_df.rent_MoI_4_br_count*units_df.weight
units_df.rent_HI_studio_count=units_df.rent_HI_studio_count*units_df.weight
units_df.rent_HI_1_br_count=units_df.rent_HI_1_br_count*units_df.weight
units_df.rent_HI_2_br_count=units_df.rent_HI_2_br_count*units_df.weight
units_df.rent_HI_3_br_count=units_df.rent_HI_3_br_count*units_df.weight
units_df.rent_HI_4_br_count=units_df.rent_HI_4_br_count*units_df.weight
units_df.own_ELI_studio_count=units_df.own_ELI_studio_count*units_df.weight
units_df.own_ELI_1_br_count=units_df.own_ELI_1_br_count*units_df.weight
units_df.own_ELI_2_br_count=units_df.own_ELI_2_br_count*units_df.weight
units_df.own_ELI_3_br_count=units_df.own_ELI_3_br_count*units_df.weight
units_df.own_ELI_4_br_count=units_df.own_ELI_4_br_count*units_df.weight
units_df.own_VLI_studio_count=units_df.own_VLI_studio_count*units_df.weight
units_df.own_VLI_1_br_count=units_df.own_VLI_1_br_count*units_df.weight
units_df.own_VLI_2_br_count=units_df.own_VLI_2_br_count*units_df.weight
units_df.own_VLI_3_br_count=units_df.own_VLI_3_br_count*units_df.weight
units_df.own_VLI_4_br_count=units_df.own_VLI_4_br_count*units_df.weight
units_df.own_LI_studio_count=units_df.own_LI_studio_count*units_df.weight
units_df.own_LI_1_br_count=units_df.own_LI_1_br_count*units_df.weight
units_df.own_LI_2_br_count=units_df.own_LI_2_br_count*units_df.weight
units_df.own_LI_3_br_count=units_df.own_LI_3_br_count*units_df.weight
units_df.own_LI_4_br_count=units_df.own_LI_4_br_count*units_df.weight
units_df.own_MI_studio_count=units_df.own_MI_studio_count*units_df.weight
units_df.own_MI_1_br_count=units_df.own_MI_1_br_count*units_df.weight
units_df.own_MI_2_br_count=units_df.own_MI_2_br_count*units_df.weight
units_df.own_MI_3_br_count=units_df.own_MI_3_br_count*units_df.weight
units_df.own_MI_4_br_count=units_df.own_MI_4_br_count*units_df.weight
units_df.own_MoI_studio_count=units_df.own_MoI_studio_count*units_df.weight
units_df.own_MoI_1_br_count=units_df.own_MoI_1_br_count*units_df.weight
units_df.own_MoI_2_br_count=units_df.own_MoI_2_br_count*units_df.weight
units_df.own_MoI_3_br_count=units_df.own_MoI_3_br_count*units_df.weight
units_df.own_MoI_4_br_count=units_df.own_MoI_4_br_count*units_df.weight
units_df.own_HI_studio_count=units_df.own_HI_studio_count*units_df.weight
units_df.own_HI_1_br_count=units_df.own_HI_1_br_count*units_df.weight
units_df.own_HI_2_br_count=units_df.own_HI_2_br_count*units_df.weight
units_df.own_HI_3_br_count=units_df.own_HI_3_br_count*units_df.weight
units_df.own_HI_4_br_count=units_df.own_HI_4_br_count*units_df.weight
units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,...,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_monthly_cost,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count
0,H,8508,110.00,4.00,2.00,nan,2300.00,nan,nan,1.00,nan,975000.00,nan,nan,105000.00,7.00,42.00,Santa Clara CA,NaN,NaN,NaN,NaN,NaN,NaN,Santa Clara,551.25,630.00,787.50,913.75,1040.00,918.75,1050.00,1312.50,1522.50,1732.50,1325.00,1515.00,1892.50,2196.25,2498.75,1837.50,2100.00,2625.00,3045.00,3465.00,2205.00,2520.00,3150.00,3653.75,4157.50,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,H,7502,180.00,1.00,9.00,nan,nan,nan,nan,nan,4.00,800000.00,nan,nan,nan,nan,nan,San Francisco CA,NaN,NaN,NaN,NaN,NaN,NaN,San Francisco,583.75,666.25,832.50,966.25,1100.00,971.25,1110.00,1387.50,1610.00,1832.50,1555.00,1776.25,2220.00,2576.25,2931.25,1802.50,2060.00,2575.00,2987.50,3398.75,2162.50,2472.50,3090.00,3585.00,4078.75,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,H,8105,75.00,2.00,7.00,nan,nan,1500.00,nan,3.00,nan,nan,1560.00,47.00,39600.00,4.00,nan,San Mateo CA,NaN,NaN,NaN,NaN,NaN,NaN,San Mateo,583.75,666.25,832.50,966.25,1100.00,971.25,1110.00,1387.50,1610.00,1832.50,1555.00,1776.25,2220.00,2576.25,2931.25,1802.50,2060.00,2575.00,2987.50,3398.75,2162.50,2472.50,3090.00,3585.00,4078.75,...,0.00,75.00,0.00,0.00,0.00,0.00,75.00,0.00,0.00,0.00,0.00,75.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,H,1305,76.00,4.00,2.00,nan,4800.00,nan,nan,1.00,nan,1000000.00,nan,nan,269000.00,4.00,68.00,Contra Costa CA,NaN,NaN,NaN,NaN,NaN,NaN,Contra Costa,491.25,561.25,701.25,813.75,926.25,818.75,935.00,1168.75,1356.25,1543.75,1143.75,1307.50,1633.75,1896.25,2157.50,1636.25,1870.00,2337.50,2711.25,3085.00,1963.75,2243.75,2805.00,3253.75,3702.50,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,H,2901,12.00,2.00,1.00,nan,690.00,nan,nan,1.00,nan,30000.00,nan,nan,65200.00,5.00,6.00,Kern CA,NaN,NaN,NaN,NaN,NaN,NaN,Kern,303.75,347.50,433.75,503.75,573.75,507.50,580.00,723.

In [20]:
units_df[units_df.rent_HI_4_br_count>0]

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,...,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_monthly_cost,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count
410,H,7111,93.00,4.00,2.00,nan,nan,2700.00,nan,3.00,nan,nan,3108.00,41.00,92000.00,2.00,nan,San Bernardino CA,NaN,NaN,NaN,NaN,NaN,NaN,San Bernardino,352.50,402.50,502.50,583.75,663.75,586.25,670.00,837.50,972.50,1106.25,938.75,1072.50,1340.00,1555.00,1770.00,1107.50,1266.25,1582.50,1836.25,2088.75,1328.75,1518.75,1898.75,2202.50,2506.25,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,93.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2874,H,3728,122.00,4.00,5.00,nan,nan,3500.00,nan,3.00,nan,nan,3680.00,93.00,47500.00,5.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,443.75,506.25,632.50,733.75,835.00,738.75,843.75,1053.75,1222.50,1391.25,1181.25,1350.00,1686.25,1956.25,2226.25,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,122.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7411,H,3720,113.00,4.00,2.00,nan,nan,2700.00,nan,3.00,nan,nan,2978.00,35.00,100700.00,2.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,443.75,506.25,632.50,733.75,835.00,738.75,843.75,1053.75,1222.50,1391.25,1181.25,1350.00,1686.25,1956.25,2226.25,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,113.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7721,H,3705,43.00,4.00,2.00,nan,nan,2600.00,nan,3.00,nan,nan,3168.00,23.00,167500.00,1.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,443.75,506.25,632.50,733.75,835.00,738.75,843.75,1053.75,1222.50,1391.25,1181.25,1350.00,1686.25,1956.25,2226.25,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,43.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9206,H,3726,129.00,4.00,2.00,nan,nan,3500.00,nan,3.00,nan,nan,3780.00,11.00,403000.00,3.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,443.75,506.25,632.5

# Part D1. Add dummy variables for vacant "for-rent," vacant "for-sale," and recently moved in

In [21]:
# create dummy variables for vacant for rent, vacant for sale, rented in the last year, and sold in the last year
units_df["vacant_for_rent"]=np.where(units_df['vacancy_status']==1,1,0)
units_df["vacant_for_sale"]=np.where(units_df['vacancy_status']==3,1,0)
units_df["moved_last_year"]=np.where(units_df['moved_in']==1,1,0)
units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,...,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_monthly_cost,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count,vacant_for_rent,vacant_for_sale,moved_last_year
0,H,8508,110.00,4.00,2.00,nan,2300.00,nan,nan,1.00,nan,975000.00,nan,nan,105000.00,7.00,42.00,Santa Clara CA,NaN,NaN,NaN,NaN,NaN,NaN,Santa Clara,551.25,630.00,787.50,913.75,1040.00,918.75,1050.00,1312.50,1522.50,1732.50,1325.00,1515.00,1892.50,2196.25,2498.75,1837.50,2100.00,2625.00,3045.00,3465.00,2205.00,2520.00,3150.00,3653.75,4157.50,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0
1,H,7502,180.00,1.00,9.00,nan,nan,nan,nan,nan,4.00,800000.00,nan,nan,nan,nan,nan,San Francisco CA,NaN,NaN,NaN,NaN,NaN,NaN,San Francisco,583.75,666.25,832.50,966.25,1100.00,971.25,1110.00,1387.50,1610.00,1832.50,1555.00,1776.25,2220.00,2576.25,2931.25,1802.50,2060.00,2575.00,2987.50,3398.75,2162.50,2472.50,3090.00,3585.00,4078.75,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0
2,H,8105,75.00,2.00,7.00,nan,nan,1500.00,nan,3.00,nan,nan,1560.00,47.00,39600.00,4.00,nan,San Mateo CA,NaN,NaN,NaN,NaN,NaN,NaN,San Mateo,583.75,666.25,832.50,966.25,1100.00,971.25,1110.00,1387.50,1610.00,1832.50,1555.00,1776.25,2220.00,2576.25,2931.25,1802.50,2060.00,2575.00,2987.50,3398.75,2162.50,2472.50,3090.00,3585.00,4078.75,...,0.00,0.00,0.00,75.00,0.00,0.00,0.00,0.00,75.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0
3,H,1305,76.00,4.00,2.00,nan,4800.00,nan,nan,1.00,nan,1000000.00,nan,nan,269000.00,4.00,68.00,Contra Costa CA,NaN,NaN,NaN,NaN,NaN,NaN,Contra Costa,491.25,561.25,701.25,813.75,926.25,818.75,935.00,1168.75,1356.25,1543.75,1143.75,1307.50,1633.75,1896.25,2157.50,1636.25,1870.00,2337.50,2711.25,3085.00,1963.75,2243.75,2805.00,3253.75,3702.50,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0
4,H,2901,12.00,2.00,1.00,nan,690.00,nan,nan,1.00,nan,30000.00,nan,nan,65200.00,5.00,6.00,Kern CA,NaN,NaN,NaN,NaN,NaN,NaN,Kern,303.75,347.50,433.75,503.75,573.75,507.50,580.00,723.75,840.00,956.25,811.25,926.25,1157.50,1343.75

In [22]:
#filter for only currently listed or recently moved into units
recent_mask = (units_df.vacant_for_rent==1)|(units_df.vacant_for_sale==1)|(units_df.moved_last_year==1)
recent_units_df = units_df[recent_mask]
recent_units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,...,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_monthly_cost,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count,vacant_for_rent,vacant_for_sale,moved_last_year
5,H,8511,71.00,3.00,2.00,nan,2700.00,nan,nan,1.00,nan,850000.00,nan,nan,80000.00,1.00,68.00,Santa Clara CA,NaN,NaN,NaN,NaN,NaN,NaN,Santa Clara,551.25,630.00,787.50,913.75,1040.00,918.75,1050.00,1312.50,1522.50,1732.50,1325.00,1515.00,1892.50,2196.25,2498.75,1837.50,2100.00,2625.00,3045.00,3465.00,2205.00,2520.00,3150.00,3653.75,4157.50,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,1
9,H,3742,113.00,2.00,4.00,nan,nan,810.00,nan,3.00,nan,nan,820.00,29.00,33800.00,1.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,443.75,506.25,632.50,733.75,835.00,738.75,843.75,1053.75,1222.50,1391.25,1181.25,1350.00,1686.25,1956.25,2226.25,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25,...,0.00,0.00,0.00,113.00,0.00,0.00,0.00,0.00,113.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,1
10,H,7306,241.00,4.00,2.00,nan,nan,1100.00,nan,3.00,nan,nan,1353.00,38.00,43100.00,1.00,nan,San Diego CA,NaN,NaN,NaN,NaN,NaN,NaN,San Diego,422.50,482.50,602.50,700.00,796.25,703.75,803.75,1003.75,1165.00,1325.00,1125.00,1285.00,1606.25,1863.75,2121.25,1328.75,1517.50,1897.50,2201.25,2505.00,1593.75,1822.50,2277.50,2642.50,3006.25,...,241.00,0.00,0.00,0.00,0.00,241.00,0.00,0.00,0.00,0.00,241.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,1
15,H,7313,59.00,3.00,5.00,nan,nan,1600.00,nan,3.00,nan,nan,1690.00,16.00,130000.00,1.00,nan,San Diego CA,NaN,NaN,NaN,NaN,NaN,NaN,San Diego,422.50,482.50,602.50,700.00,796.25,703.75,803.75,1003.75,1165.00,1325.00,1125.00,1285.00,1606.25,1863.75,2121.25,1328.75,1517.50,1897.50,2201.25,2505.00,1593.75,1822.50,2277.50,2642.50,3006.25,...,0.00,0.00,0.00,0.00,59.00,0.00,0.00,0.00,0.00,59.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,1
16,H,3734,52.00,1.00,8.00,nan,nan,900.00,nan,3.00,nan,nan,980.00,44.00,27000.00,1.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,443.75,506.25,632.50,733.75,835.00,738.75,843.75,1053.75,1222.50,1391.25,1181.25,13

# Part D2. Aggregating at the PUMA level

In [23]:
# for all units
puma_sums_df = units_df.groupby("PUMA").sum()
puma_counts_df = puma_sums_df[["rent_ELI_studio_count","rent_ELI_1_br_count",
    "rent_ELI_2_br_count",
    "rent_ELI_3_br_count",
    "rent_ELI_4_br_count",
    "rent_VLI_studio_count",
    "rent_VLI_1_br_count",
    "rent_VLI_2_br_count",
    "rent_VLI_3_br_count",
    "rent_VLI_4_br_count",
    "rent_LI_studio_count",
    "rent_LI_1_br_count",
    "rent_LI_2_br_count",
    "rent_LI_3_br_count",
    "rent_LI_4_br_count",
    "rent_MI_studio_count",
    "rent_MI_1_br_count",
    "rent_MI_2_br_count",
    "rent_MI_3_br_count",
    "rent_MI_4_br_count",
    "rent_MoI_studio_count",
    "rent_MoI_1_br_count",
    "rent_MoI_2_br_count",
    "rent_MoI_3_br_count",
    "rent_MoI_4_br_count",
    "rent_HI_studio_count",
    "rent_HI_1_br_count",
    "rent_HI_2_br_count",
    "rent_HI_3_br_count",
    "rent_HI_4_br_count",
    "own_ELI_studio_count",
    "own_ELI_1_br_count",
    "own_ELI_2_br_count",
    "own_ELI_3_br_count",
    "own_ELI_4_br_count",
    "own_VLI_studio_count",
    "own_VLI_1_br_count",
    "own_VLI_2_br_count",
    "own_VLI_3_br_count",
    "own_VLI_4_br_count",
    "own_LI_studio_count",
    "own_LI_1_br_count",
    "own_LI_2_br_count",
    "own_LI_3_br_count",
    "own_LI_4_br_count",
    "own_MI_studio_count",
    "own_MI_1_br_count",
    "own_MI_2_br_count",
    "own_MI_3_br_count",
    "own_MI_4_br_count",
    "own_MoI_studio_count",
    "own_MoI_1_br_count",
    "own_MoI_2_br_count",
    "own_MoI_3_br_count",
    "own_MoI_4_br_count",
    "own_HI_studio_count",
    "own_HI_1_br_count",
    "own_HI_2_br_count",
    "own_HI_3_br_count",
    "own_HI_4_br_count"]].copy()
puma_counts_df

,rent_ELI_studio_count,rent_ELI_1_br_count,rent_ELI_2_br_count,rent_ELI_3_br_count,rent_ELI_4_br_count,rent_VLI_studio_count,rent_VLI_1_br_count,rent_VLI_2_br_count,rent_VLI_3_br_count,rent_VLI_4_br_count,rent_LI_studio_count,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count
PUMA,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
101,662.00,1729.00,1125.00,302.00,573.00,2516.00,4367.00,2687.00,688.00,707.00,3519.00,9875.00,6361.00,1363.00,1016.00,4154.00,11234.00,9411.00,2129.00,1187.00,4612.00,11665.00,9546.00,2506.00,1347.00,355.00,282.00,0.00,168.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,52.00,0.00,0.00,0.00
102,1625.00,4038.00,2725.00,1310.00,383.00,5070.00,12854.00,7599.00,2874.00,950.00,7435.00,20431.00,12042.00,4666.00,1222.00,9032.00,23269.00,15295.00,5416.00,1533.00,9108.00,24316.00,16155.00,5494.00,1533.00,342.00,467.00,68.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,67.00,0.00,0.00,0.00,0.00,67.00,0.00,0.00,0.00,137.00,67.00,320.00,0.00,0.00,206.00,67.00,320.00,0.00,0.00,216.00,411.00,0.00,0.00
103,116.00,1229.00,716.00,477.00,57.00,235.00,2796.00,3093.00,853.00,57.00,977.00,7086.00,6038.00,1813.00,57.00,1032.00,7591.00,6856.00,1943.00,614.00,1032.00,7591.00,7012.00,1943.00,724.00,279.00,227.00,256.00,385.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,129.00,56.00,0.00,0.00,0.00,129.00,96.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
104,240.00,936.00,2826.00,1268.00,521.00,1710.00,4233.00,7803.00,2990.00,939.00,2069.00,5233.00,11106.00,4917.00,1097.00,2353.00,5281.00,11106.00,5249.00,1097.00,2353.00,5281.00,11106.00,5249.00,1097.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
105,0.00,1164.00,552.00,222.00,169.00,416.00,3574.00,4095.00,2484.00,567.00,982.00,7373.00,12332.00,4591.00,1133.00,1557.00,7999.00,13865.00,5239.00,1291.00,1557.00,7999.00,13993.00,5239.00,1418.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,131.00,0.00,0.00,0.00,334.00,131.00,0.00,0.00,0.00,410.00,185.00,0.00,0.00,0.00,410.00,185.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11103,155.00,315.00,504.00,476.00,335.00,1083.00,2112.00,2436.00,1277.00,553.00,1521.00,4665.00,7946.00,4575.00,2219.00,1521.00,5112.00,8841.00,5270.00,2362.00,1521.00,5112.00,8949.00,5402.00,2699.00,0.00,0.00,266.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,246.00,0.00,0.00,0.00,199.00,311.00,0.00,0.00,0.00,199.00,311.00,0.00,0.00,0.00,0.00,0.00,0.00
11104,0.00,701.00,320.00,762.00,0.00,690.00,1927.00,2218.00,1730.00,213.00,1992.00,4998.00,6000.00,3849.00,851.00,2357.00,5551.00,7125.00,4850.00,916.00,2357.00,5551.00,7125.00,4909.00,916.00,191.00,65.00,0.

In [24]:
#export it
puma_counts_df.to_csv("weighted_puma_counts.csv")

In [25]:
# for recent units
recent_puma_sums_df = recent_units_df.groupby("PUMA").sum()
recent_puma_counts_df = recent_puma_sums_df[["rent_ELI_studio_count","rent_ELI_1_br_count",
    "rent_ELI_2_br_count",
    "rent_ELI_3_br_count",
    "rent_ELI_4_br_count",
    "rent_VLI_studio_count",
    "rent_VLI_1_br_count",
    "rent_VLI_2_br_count",
    "rent_VLI_3_br_count",
    "rent_VLI_4_br_count",
    "rent_LI_studio_count",
    "rent_LI_1_br_count",
    "rent_LI_2_br_count",
    "rent_LI_3_br_count",
    "rent_LI_4_br_count",
    "rent_MI_studio_count",
    "rent_MI_1_br_count",
    "rent_MI_2_br_count",
    "rent_MI_3_br_count",
    "rent_MI_4_br_count",
    "rent_MoI_studio_count",
    "rent_MoI_1_br_count",
    "rent_MoI_2_br_count",
    "rent_MoI_3_br_count",
    "rent_MoI_4_br_count",
    "rent_HI_studio_count",
    "rent_HI_1_br_count",
    "rent_HI_2_br_count",
    "rent_HI_3_br_count",
    "rent_HI_4_br_count",
    "own_ELI_studio_count",
    "own_ELI_1_br_count",
    "own_ELI_2_br_count",
    "own_ELI_3_br_count",
    "own_ELI_4_br_count",
    "own_VLI_studio_count",
    "own_VLI_1_br_count",
    "own_VLI_2_br_count",
    "own_VLI_3_br_count",
    "own_VLI_4_br_count",
    "own_LI_studio_count",
    "own_LI_1_br_count",
    "own_LI_2_br_count",
    "own_LI_3_br_count",
    "own_LI_4_br_count",
    "own_MI_studio_count",
    "own_MI_1_br_count",
    "own_MI_2_br_count",
    "own_MI_3_br_count",
    "own_MI_4_br_count",
    "own_MoI_studio_count",
    "own_MoI_1_br_count",
    "own_MoI_2_br_count",
    "own_MoI_3_br_count",
    "own_MoI_4_br_count",
    "own_HI_studio_count",
    "own_HI_1_br_count",
    "own_HI_2_br_count",
    "own_HI_3_br_count",
    "own_HI_4_br_count"]].copy()
recent_puma_counts_df

,rent_ELI_studio_count,rent_ELI_1_br_count,rent_ELI_2_br_count,rent_ELI_3_br_count,rent_ELI_4_br_count,rent_VLI_studio_count,rent_VLI_1_br_count,rent_VLI_2_br_count,rent_VLI_3_br_count,rent_VLI_4_br_count,rent_LI_studio_count,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count
PUMA,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
101,0.00,161.00,356.00,45.00,204.00,404.00,733.00,1048.00,45.00,338.00,959.00,3351.00,2124.00,547.00,537.00,1458.00,4066.00,4089.00,622.00,651.00,1487.00,4448.00,4089.00,999.00,651.00,0.00,58.00,0.00,97.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
102,250.00,466.00,481.00,0.00,35.00,2041.00,2807.00,1283.00,241.00,83.00,2591.00,4216.00,2220.00,975.00,186.00,2869.00,4840.00,3301.00,1057.00,369.00,2908.00,5521.00,3772.00,1094.00,369.00,342.00,225.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,74.00,0.00,0.00,0.00
103,0.00,0.00,65.00,0.00,0.00,0.00,447.00,651.00,264.00,0.00,239.00,2353.00,1834.00,540.00,0.00,294.00,2575.00,2106.00,605.00,178.00,294.00,2575.00,2198.00,605.00,288.00,211.00,62.00,0.00,254.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
104,170.00,155.00,516.00,243.00,481.00,361.00,1509.00,1940.00,533.00,481.00,361.00,2058.00,3119.00,1273.00,607.00,506.00,2058.00,3119.00,1494.00,607.00,506.00,2058.00,3119.00,1494.00,607.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
105,0.00,184.00,0.00,59.00,0.00,201.00,464.00,1066.00,212.00,0.00,361.00,1960.00,3491.00,392.00,0.00,572.00,2409.00,4243.00,638.00,86.00,572.00,2409.00,4243.00,638.00,213.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11103,155.00,49.00,0.00,0.00,0.00,508.00,326.00,386.00,257.00,0.00,508.00,1598.00,2004.00,1126.00,785.00,508.00,1977.00,2356.00,1126.00,928.00,508.00,1977.00,2464.00,1258.00,928.00,0.00,0.00,266.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
11104,0.00,117.00,0.00,0.00,0.00,105.00,548.00,185.00,410.00,0.00,715.00,2154.00,1267.00,604.00,123.00,914.00,2347.00,1267.00,701.00,123.00,914.00,2347.00,1267.00,760.00,123.00,191.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
11105,0.00,0.00,35.0

In [26]:
#export it
recent_puma_counts_df.to_csv("weighted_recent_puma_counts.csv")